<a target="_blank" href="https://colab.research.google.com/github/Nick24-hub/Advanced-Topics-in-Neural-Networks-Template-2023/blob/main/Lab07/Solution/assignment6.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import torch

In [2]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
        # For multi-gpu workstations, PyTorch will use the first available GPU (cuda:0), unless specified otherwise
        # (cuda:1).
    if torch.backends.mps.is_available():
        return torch.device('mos')
    return torch.device('cpu')

In [3]:
class Handmade_conv2d_implementation(torch.nn.Module):
    def __init__(self, weights):
        super(Handmade_conv2d_implementation, self).__init__()
        self.weights = weights

    def forward(self, x):
        # Extracting dimensions
        batch_size, in_channels, in_height, in_width = x.shape
        out_channels, _, kernel_height, kernel_width = self.weights.shape

        # Output dimensions
        out_height = in_height - kernel_height + 1
        out_width = in_width - kernel_width + 1

        # Reshape input and weights for matrix multiplication
        x_reshaped = x.unfold(2, kernel_height, 1).unfold(3, kernel_width, 1)
        x_reshaped = x_reshaped.permute(0, 2, 3, 1, 4, 5).contiguous().view(batch_size, out_height, out_width, -1)

        w_reshaped = self.weights.view(out_channels, -1).t()

        # Perform matrix multiplication and reshape back to the output shape
        out = torch.matmul(x_reshaped, w_reshaped).view(batch_size, out_height, out_width, out_channels)
        out = out.permute(0, 3, 1, 2).contiguous()

        return out

In [4]:
device = get_default_device()
print(device)

inp = torch.randn(1, 3, 10, 12).to(device) # Input image

# kernel of size 4x5 , with 3 input channels and 2 output channels
w = torch.randn(2, 3, 4, 5).to(device) # Conv weights

# Your implementation. Can be made differently, like only passing the kernel size for example
custom_conv2d_layer = Handmade_conv2d_implementation(weights = w)
      
out = custom_conv2d_layer(inp)
      
print ((torch.nn.functional.conv2d(inp,w) - out).abs().max())

cuda
tensor(2.8610e-06, device='cuda:0')
